In [1]:
from utils.pre_MEDS import generate_meds_preprocessed
from utils.synthetic_generator import generate_synthetic_dataset
import os

os.makedirs("input", exist_ok=True)
os.makedirs("intermediate", exist_ok=True)
os.makedirs("output", exist_ok=True)

df_input = generate_synthetic_dataset(3, output_csv="input/syn_data.csv")
df_intermediate = generate_meds_preprocessed(df_input, output_path="intermediate")

In [2]:
from MEDS_transforms.runner import main
import shutil

MEDS_ETL_OUTPUT = "output"
shutil.rmtree(MEDS_ETL_OUTPUT)

main([
    "pkg://MEDS_extract.configs._extract.yaml",
    "--overrides",
    "input_dir=intermediate",
    "output_dir=output",
    "event_conversion_config_fp=MESSY.yaml",
    "dataset.name=Neurovasc",
    "dataset.version=1.0",
])

0

In [3]:
from meds2rdf import MedsRDFConverter

# Initialize the converter with the path to your MEDS dataset directory
converter = MedsRDFConverter("output")

# Convert the dataset into an RDF graph
graph = converter.convert(
    include_splits=True,
)

OUTPUT_DIR = "output/rdf"

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Serialize the graph to different formats
graph.serialize(destination=f"{OUTPUT_DIR}/output_dataset.ttl", format="turtle")
graph.serialize(destination=f"{OUTPUT_DIR}/output_dataset.xml", format="xml")
graph.serialize(destination=f"{OUTPUT_DIR}/output_dataset.nt", format="nt")

print("Conversion complete! RDF files saved.")

Conversion complete! RDF files saved.


/home/amarf/workspace/heka-team/neurovasc_on_meds/.venv/lib/python3.12/site-packages/rdflib/plugins/serializers/nt.py:39: UserWarning: NTSerializer always uses UTF-8 encoding. Given encoding was: None
  warnings.warn(


In [4]:
from pyshacl import validate

data_graph = "output/rdf/output_dataset.ttl"
shacl_graph = "https://raw.githubusercontent.com/TeamHeKA/meds-ontology/refs/heads/main/shacl/meds-shapes.ttl"

output_graph = validate(data_graph, shacl_graph=shacl_graph, advanced=True)

print(output_graph[2])

Validation Report
Conforms: True

